In [113]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer


In [114]:
PATH_TO_DATA = '/Users/dariabakshandaeva/Documents/data'

In [115]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA) if file.endswith('jsonlines')]

In [116]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [117]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

теперь подготовим текст к использованию TermExtractor (избавимся от пунктуации и превратим в строки; стоп-слова убирать не нужно)

In [222]:
from string import punctuation
punct = punctuation+'«»—…“”*№–'

def words(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return words

data['content_norm'] = data['content'].apply(words)

data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [218]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from rutermextract import TermExtractor
term_extractor = TermExtractor()


def extr(text):
    words = [term.normalized for term in term_extractor(text)]
    
    return words


data['keywords_extr'] = data['content_norm_str'].apply(extr)


In [219]:
data['keywords_extr'][:3]

0    [ольга васильева, новые фгосы, историческое ис...
1    [красота, ваша красота, руина, куда, глаза, са...
2    [пепеляев, юзефович, якутия, места, книга, нач...
Name: keywords_extr, dtype: object

In [221]:
evaluate(data['keywords'], data['keywords_extr'].apply(lambda x: [x[0] for x in Counter(x).most_common(5)]))

Precision -  0.18
Recall -  0.17
F1 -  0.16
Jaccard -  0.1


неплохо, но baseline не побило :(

Попробуем NLTK Rake

In [239]:
content_list = pd.Series.tolist(data['content'])

from rake_nltk import Metric, Rake

r = Rake(language='russian', stopwords=stopwords.words('russian'), min_length=1, max_length=2) #берем униграммы и биграммы

r.extract_keywords_from_text(content_list[0])
r.get_ranked_phrases()

['число книг',
 'цели создания',
 'участников событий',
 'увязываю это',
 'точки зрения',
 'существующему стандарту',
 'станет понятно',
 'средней школы',
 'сохранена структура',
 'сонме романов',
 'совокупность требований',
 'собственное решение',
 'собрание должно',
 'своей страничке',
 'самое важное',
 'редакции фгоса',
 'развивают память',
 'против которого',
 'пришло сообщение',
 'прежней редакции',
 'полностью отсутствовало',
 'показала разницу',
 'позицией министерства',
 'подробных пункта',
 'поверхностного разговора',
 'оценивать содержащуюся',
 'основного общего',
 'основе план',
 'однако ученики',
 'начальной школы',
 'науки рф',
 'министерстве образования',
 'место создания',
 'любому родителю',
 'которую развязали',
 'историческом источнике',
 'интересно знать',
 'изучении событий',
 'дополнительной информации',
 'го класса',
 'вчерашнем заседании',
 'включая литературу',
 '– заявила',
 '– вспомните',
 'школе ».',
 'списку ».',
 'основной школы',
 'основной мысли',
 'спосо

Униграммы и биграммы в большинстве своем нерелевантны, плохого качества